In [15]:
# The 6.00 Word Game

import random
import string
import time

VOWELS = 'aeiou'
CONSONANTS = 'bcdfghjklmnpqrstvwxyz'
HAND_SIZE = 7

SCRABBLE_LETTER_VALUES = {
    'a': 1, 'b': 3, 'c': 3, 'd': 2, 'e': 1, 'f': 4, 'g': 2, 'h': 4, 'i': 1, 'j': 8, 'k': 5, 'l': 1, 'm': 3, 'n': 1, 'o': 1, 'p': 3, 'q': 10, 'r': 1, 's': 1, 't': 1, 'u': 1, 'v': 4, 'w': 4, 'x': 8, 'y': 4, 'z': 10
}

# -----------------------------------
# Helper code

WORDLIST_FILENAME = "words.txt"

def loadWords():
    """
    Returns a list of valid words. Words are strings of lowercase letters.
    
    Depending on the size of the word list, this function may
    take a while to finish.
    """
    print("Loading word list from file...")
    # inFile: file
    inFile = open(WORDLIST_FILENAME, 'r')
    # wordList: list of strings
    wordList = []
    for line in inFile:
        wordList.append(line.strip().lower())
    print("  ", len(wordList), "words loaded.")
    return wordList

def getFrequencyDict(sequence):
    """
    Returns a dictionary where the keys are elements of the sequence
    and the values are integer counts, for the number of times that
    an element is repeated in the sequence.

    sequence: string or list
    return: dictionary
    """
    # freqs: dictionary (element_type -> int)
    freq = {}
    for x in sequence:
        freq[x] = freq.get(x,0) + 1
    return freq
	

# (end of helper code)
# -----------------------------------

#
# Problem #1: Scoring a word
#
def getWordScore(word, n):
    """
    Returns the score for a word. Assumes the word is a valid word.

    The score for a word is the sum of the points for letters in the
    word, multiplied by the length of the word, PLUS 50 points if all n
    letters are used on the first turn.

    Letters are scored as in Scrabble; A is worth 1, B is worth 3, C is
    worth 3, D is worth 2, E is worth 1, and so on (see SCRABBLE_LETTER_VALUES)

    word: string (lowercase letters)
    n: integer (HAND_SIZE; i.e., hand size required for additional points)
    returns: int >= 0
    """
    
    count = 0

    for i in list(word):
        for key, value in SCRABBLE_LETTER_VALUES.items():
            if key == i:
                count += value

    count*=len(word)

    if len(word) == n:
        count+= 50

    return count



#
# Problem #2: Make sure you understand how this function works and what it does!
#
def displayHand(hand):
    """
    Displays the letters currently in the hand.

    For example:
    >>> displayHand({'a':1, 'x':2, 'l':3, 'e':1})
    Should print out something like:
       a x x l l l e
    The order of the letters is unimportant.

    hand: dictionary (string -> int)
    """
    for letter in hand.keys():
        for j in range(hand[letter]):
             print(letter,end=" ")       # print all on the same line
    print()                             # print an empty line

#
# Problem #2: Make sure you understand how this function works and what it does!
#
def dealHand(n):
    """
    Returns a random hand containing n lowercase letters.
    At least n/3 the letters in the hand should be VOWELS.

    Hands are represented as dictionaries. The keys are
    letters and the values are the number of times the
    particular letter is repeated in that hand.

    n: int >= 0
    returns: dictionary (string -> int)
    """
    hand={}
    numVowels = n // 3
    
    for i in range(numVowels):
        x = VOWELS[random.randrange(0,len(VOWELS))]
        hand[x] = hand.get(x, 0) + 1
        
    for i in range(numVowels, n):    
        x = CONSONANTS[random.randrange(0,len(CONSONANTS))]
        hand[x] = hand.get(x, 0) + 1
        
    return hand

#
# Problem #2: Update a hand by removing letters
#
def updateHand(hand, word):
    """
    Assumes that 'hand' has all the letters in word.
    In other words, this assumes that however many times
    a letter appears in 'word', 'hand' has at least as
    many of that letter in it. 

    Updates the hand: uses up the letters in the given word
    and returns the new hand, without those letters in it.

    Has no side effects: does not modify hand.

    word: string
    hand: dictionary (string -> int)    
    returns: dictionary (string -> int)
    """
    updated_hand = hand
    
    for i in list(word):
        for key, value in updated_hand.items():
            if i == key:
                updated_hand[key] = value - 1

    return updated_hand



#
# Problem #3: Test word validity
#
def isValidWord(word, hand, wordList):
    """
    Returns True if word is in the wordList and is entirely
    composed of letters in the hand. Otherwise, returns False.

    Does not mutate hand or wordList.
   
    word: string
    hand: dictionary (string -> int)
    wordList: list of lowercase strings
    """
    try:
        for key, value in getFrequencyDict(word).items():
            assert value <= hand[key]
        assert word in wordList
    except:
        return False
    else:
        return True


#
# Problem #4: Playing a hand
#

def calculateHandlen(hand):
    """ 
    Returns the length (number of letters) in the current hand.
    
    hand: dictionary (string-> int)
    returns: integer
    """
    count = 0
    
    for key, value in hand.items():
        count += value

    return count



def playHand(hand, wordList, n):
    """
    Allows the user to play the given hand, as follows:

    * The hand is displayed.
    * The user may input a word or a single period (the string ".") 
      to indicate they're done playing
    * Invalid words are rejected, and a message is displayed asking
      the user to choose another word until they enter a valid word or "."
    * When a valid word is entered, it uses up letters from the hand.
    * After every valid word: the score for that word is displayed,
      the remaining letters in the hand are displayed, and the user
      is asked to input another word.
    * The sum of the word scores is displayed when the hand finishes.
    * The hand finishes when there are no more unused letters or the user
      inputs a "."

      hand: dictionary (string -> int)
      wordList: list of lowercase strings
      n: integer (HAND_SIZE; i.e., hand size required for additional points)
      
    """
    # BEGIN PSEUDOCODE 
    # Keep track of the total score
    
    score = 0
    n = HAND_SIZE
    
    # As long as there are still letters left in the hand:
    
    while calculateHandlen(hand) > 0:
    
        # Display the hand
        print('Your current hand is: ')
        displayHand(hand)
        print('')
        
        # Ask user for input
        
        print('Enter word, or a "." to indicate that you are finished:')
        word = input()
        
        # If the input is a single period:
        
        if word == '.':
        
            # End the game (break out of the loop)
            
            break

            
        # Otherwise (the input is not a single period):
        
        else:
        
            # If the word is not valid:
            
            if isValidWord(word, hand, wordList) == False:
              
            
                # Reject invalid word (print a message followed by a blank line)
                
                print('Your word is invalid.')
                print('')
            

            # Otherwise (the word is valid):
            
            else:
              
                # Tell the user how many points the word earned, and the updated total score, in one line followed by a blank line
                
                score += getWordScore(word, n)
                print('You have earned '+str(getWordScore(word, calculateHandlen(hand)))+' points.')
                print('Your new score is: '+str(score))
                print('')
                
                # Update the hand 
                updateHand(hand, word)

    # Game is over (user entered a '.' or ran out of letters), so tell user the total score
    
    print('Thank you for playing. Your total score is: '+str(score))
    print('')
    print('')


#
#
# Computer chooses a word
#
#
def compChooseWord(hand, wordList, n):
    """
    Given a hand and a wordList, find the word that gives 
    the maximum value score, and return it.

    This word should be calculated by considering all the words
    in the wordList.

    If no words in the wordList can be made from the hand, return None.

    hand: dictionary (string -> int)
    wordList: list (string)
    n: integer (HAND_SIZE; i.e., hand size required for additional points)

    returns: string or None
    """
    # Create a new variable to store the maximum score seen so far (initially 0)
    bestScore = 0
    # Create a new variable to store the best word seen so far (initially None)  
    bestWord = None
    # For each word in the wordList
    for word in wordList:
        # If you can construct the word from your hand
        if isValidWord(word, hand, wordList):
            # find out how much making that word is worth
            score = getWordScore(word, n)
            # If the score for that word is higher than your best score
            if (score > bestScore):
                # update your best score, and best word accordingly
                bestScore = score
                bestWord = word
    # return the best word you found.
    return bestWord

#
# Computer plays a hand
#
def compPlayHand(hand, wordList, n):
    """
    Allows the computer to play the given hand, following the same procedure
    as playHand, except instead of the user choosing a word, the computer 
    chooses it.

    1) The hand is displayed.
    2) The computer chooses a word.
    3) After every valid word: the word and the score for that word is 
    displayed, the remaining letters in the hand are displayed, and the 
    computer chooses another word.
    4)  The sum of the word scores is displayed when the hand finishes.
    5)  The hand finishes when the computer has exhausted its possible
    choices (i.e. compChooseWord returns None).
 
    hand: dictionary (string -> int)
    wordList: list (string)
    n: integer (HAND_SIZE; i.e., hand size required for additional points)
    """
    # Keep track of the total score
    totalScore = 0
    # As long as there are still letters left in the hand:
    while (calculateHandlen(hand) > 0) :
        # Display the hand
        print("Current Hand: ", end=' ')
        displayHand(hand)
        # computer's word
        word = compChooseWord(hand, wordList, n)
        # If the input is a single period:
        if word == None:
            # End the game (break out of the loop)
            break
            
        # Otherwise (the input is not a single period):
        else :
            # If the word is not valid:
            if (not isValidWord(word, hand, wordList)) :
                print('This is a terrible error! I need to check my own code!')
                break
            # Otherwise (the word is valid):
            else :
                # Tell the user how many points the word earned, and the updated total score 
                score = getWordScore(word, n)
                totalScore += score
                print('"' + word + '" earned ' + str(score) + ' points. Total: ' + str(totalScore) + ' points')              
                # Update hand and show the updated hand to the user
                hand = updateHand(hand, word)
                print()
    # Game is over (user entered a '.' or ran out of letters), so tell user the total score
    print('Total score: ' + str(totalScore) + ' points.')

    
#
# Problem #6: Playing a game
#
#
def playGame(wordList):
    """
    Allow the user to play an arbitrary number of hands.
 
    1) Asks the user to input 'n' or 'r' or 'e'.
        * If the user inputs 'e', immediately exit the game.
        * If the user inputs anything that's not 'n', 'r', or 'e', keep asking them again.

    2) Asks the user to input a 'u' or a 'c'.
        * If the user inputs anything that's not 'c' or 'u', keep asking them again.

    3) Switch functionality based on the above choices:
        * If the user inputted 'n', play a new (random) hand.
        * Else, if the user inputted 'r', play the last hand again.
      
        * If the user inputted 'u', let the user play the game
          with the selected hand, using playHand.
        * If the user inputted 'c', let the computer play the 
          game with the selected hand, using compPlayHand.

    4) After the computer or user has played the hand, repeat from step 1

    wordList: list (string)
    """
    # TO DO... <-- Remove this comment when you code this function
    
    state = 1
    
    print('Welcome to the game!') 
    
    while state > 0:
        print('Enter n to deal a new hand, r to replay the last hand, or e to end game: n')
        user_input = input()

        if user_input == 'e':
            print('Thank you for playing!')
            break
        
        elif user_input == 'n':
            
            starting_hand = dealHand(HAND_SIZE)
            current_hand = starting_hand.copy()
            
            print('Enter u to have yourself play, c to have the computer play: ')
            user_input2 = input()
            
            if user_input2 == 'u':
                playHand(current_hand, wordList, HAND_SIZE)
                
            elif user_input2 =='c':
                compPlayHand(current_hand, wordList, HAND_SIZE)
            else:
                print('Invalid entry')

        elif user_input == 'r':
            
            try:
                type(starting_hand) is dict
            except:
                print('You have not played a hand yet. Please press "n" to play your first hand.')
            else:
                print('Enter u to have yourself play, c to have the computer play: ')
                user_input2 = input()
            
                if user_input2 == 'u':
                    playHand(starting_hand, wordList, HAND_SIZE)

                elif user_input2 =='c':
                    compPlayHand(starting_hand, wordList, HAND_SIZE)
                else:
                    print('Invalid entry')
        else:
            print('Your entry was invalid. Please try again.')
    

        
#
# Build data structures used for entire session and play game
#
if __name__ == '__main__':
    wordList = loadWords()
    playGame(wordList)




Loading word list from file...
   83667 words loaded.
Welcome to the game!
Enter n to deal a new hand, r to replay the last hand, or e to end game: n
r
You have not played a hand yet. Please press "n" to play your first hand.
Enter n to deal a new hand, r to replay the last hand, or e to end game: n
n
Enter u to have yourself play, c to have the computer play: 
u
Your current hand is: 
u a y p t t n 

Enter word, or a "." to indicate that you are finished:
patty
You have earned 50 points.
Your new score is: 50

Your current hand is: 
u n 

Enter word, or a "." to indicate that you are finished:
un
You have earned 54 points.
Your new score is: 54

Thank you for playing. Your total score is: 54


Enter n to deal a new hand, r to replay the last hand, or e to end game: n
r
Enter u to have yourself play, c to have the computer play: 
c
Current Hand:  u a y p t t n 
"panty" earned 50 points. Total: 50 points

Current Hand:  u t 
"ut" earned 4 points. Total: 54 points

Total score: 54 points